In [48]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions
spark = SparkSession.builder.getOrCreate()

In [9]:
silver_path = "/home/gnana/football-lakehouse/silver"
gold_path = "/home/gnana/football-lakehouse/gold"

In [11]:
players = spark.read.parquet(f"{silver_path}/players_silver")
games = spark.read.parquet(f"{silver_path}/games_silver")
appearances = spark.read.parquet(f"{silver_path}/appearances_silver")
clubs = spark.read.parquet(f"{silver_path}/clubs_silver")

In [12]:
fact_player_match = (
    apps
    .join(players, "player_id", "left")
    .join(games, "game_id", "left")
    .join(clubs, players.current_club_id == clubs.club_id, "left")
    .select(
        apps.appearance_id,
        apps.player_id,
        apps.game_id,
        players.name.alias("player_name"),
        players.position,
        games.date,
        games.season,
        clubs.name.alias("club_name"),
        apps.minutes_played,
        apps.goals,
        apps.assists,
        apps.yellow_cards,
        apps.red_cards
    )
)

In [14]:
fact_player_match.count()

1706806

In [15]:
fact_player_match.limit(5).toPandas()

,appearance_id,player_id,game_id,player_name,position,date,season,club_name,minutes_played,goals,assists,yellow_cards,red_cards
0,2211607_187245,187245,2211607,Mitchell Dijks,Defender,2012-08-05,2012,Fortuna Sittardia Combinatie,76,0,0,1,0
1,2211607_30290,30290,2211607,Kenneth Vermeer,Goalkeeper,2012-08-05,2012,Prins Hendrik Ende Desespereert Nimmer Combina...,90,0,0,0,0
2,2211607_33210,33210,2211607,Przemyslaw Tyton,Goalkeeper,2012-08-05,2012,Football Club Twente,90,0,1,1,0
3,2211607_52246,52246,2211607,Kolbeinn Sigthórsson,Attack,2012-08-05,2012,Football Club de Nantes,76,0,0,0,0
4,2211607_95755,95755,2211607,Viktor Fischer,Attack,2012-08-05,2012,Royal Antwerp Football Club,65,0,0,0,0


In [16]:
fact_player_match.write.mode("overwrite").parquet(f"{gold_path}/fact_player_match")

26/02/10 02:33:08 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [17]:
spark.read.parquet(f"{gold_path}/fact_player_match").count()

1706806

In [19]:
fact = spark.read.parquet(f"{gold_path}/fact_player_match")

In [70]:
player_kpis = (
    fact.groupBy("player_id", "player_name", "position")
        .agg(
            count("game_id").alias("matches_played"),
            sum("minutes_played").alias("total_minutes"),
            sum("goals").alias("goals"),
            sum("assists").alias("assists"),
            sum("yellow_cards").alias("yellow_cards"),
            sum("red_cards").alias("red_cards")
        )
        .withColumn("goal_involvement", round(col("goals") + col("assists"),3))
        .withColumn("goals_per_90'", round(col("goals") * 90 / col("total_minutes"),3))
        .withColumn("assists_per_90'", round(col("assists") * 90 / col("total_minutes"),3))
)

In [71]:
player_kpis.limit(10).toPandas()

,player_id,player_name,position,matches_played,total_minutes,goals,assists,yellow_cards,red_cards,goal_involvement,goals_per_90',assists_per_90'
0,26683,Mihaita Plesan,Midfield,9,537,0,0,3,0,0,0.000,0.000
1,145919,Arsen Khubulov,Attack,126,6687,14,17,28,1,31,0.188,0.229
2,46175,Milan Perendija,Defender,40,3158,3,0,8,1,3,0.085,0.000
3,43495,Sergiy Pylypchuk,Attack,27,2146,2,3,6,0,5,0.084,0.126
4,43299,Edmar Galovskyi,Midfield,98,7882,9,2,31,1,11,0.103,0.023
5,60573,Yann M'Vila,Midfield,345,27817,12,23,36,1,35,0.039,0.074
6,5495,John Utaka,Attack,78,5461,19,12,8,1,31,0.313,0.198
7,29241,Thiago Silva,Defender,421,35464,21,9,42,1,30,0.053,0.023
8,61335,Alexander Fischer,Defender,136,10586,5,9,10,1,14,0.043,0.077
9,23160,Stephan Petersen,Attack,133,8317,16,15,14,0,31,0.173,0.162


In [72]:
player_kpis.write.mode("overwrite").parquet(f"{gold_path}/player_kpis")

26/02/10 08:20:47 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [50]:
player_season_kpis = (
    fact.groupBy("season", "player_id", "player_name")
        .agg(
            count("game_id").alias("matches"),
            sum("minutes_played").alias("minutes"),
            sum("goals").alias("goals"),
            sum("assists").alias("assists")
        )
        .withColumn("goal_involvement", col("goals") + col("assists"))
)

In [56]:
player_season_kpis.show(10, truncate=False)

[Stage 74:=============================================>            (7 + 2) / 9]

+------+---------+----------------+-------+-------+-----+-------+----------------+
|season|player_id|player_name     |matches|minutes|goals|assists|goal_involvement|
+------+---------+----------------+-------+-------+-----+-------+----------------+
|2012  |50199    |Stephan Hain    |14     |397    |1    |0      |1               |
|2012  |51903    |Mikhail Komkov  |3      |190    |1    |0      |1               |
|2012  |75209    |Igor Shitov     |22     |1929   |0    |3      |3               |
|2012  |85430    |Jaba Lipartia   |26     |1181   |1    |1      |2               |
|2012  |55737    |Artur Yedigaryan|21     |1460   |0    |1      |1               |
|2012  |12379    |Marc Planus     |37     |3131   |0    |1      |1               |
|2012  |34361    |André Luiz      |18     |1578   |2    |0      |2               |
|2012  |67721    |Mohammed Rabiu  |28     |1939   |2    |4      |6               |
|2012  |182284   |Jeppe Illum     |15     |862    |0    |0      |0               |
|201

In [57]:
player_season_kpis.write.mode("overwrite").parquet(f"{gold_path}/player_season_kpis")

26/02/10 08:09:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [53]:
club_season_kpis = (
    fact.groupBy("season", "club_name")
        .agg(
            sum("goals").alias("goals"),
            sum("minutes_played").alias("minutes"),
            count("game_id").alias("total_players")
        )
        .orderBy("season")
)

In [54]:
club_season_kpis.show(10, truncate=False)

+------+-----------------------------+-----+-------+-------------+
|season|club_name                    |goals|minutes|total_players|
+------+-----------------------------+-----+-------+-------------+
|2012  |Bologna Football Club 1909   |66   |35343  |522          |
|2012  |Burnley FC                   |41   |29333  |401          |
|2012  |Lille Olympique Sporting Club|42   |37787  |469          |
|2012  |SD Huesca                    |4    |17562  |216          |
|2012  |Le Havre Athletic Club       |22   |5130   |65           |
|2012  |Venezia Football Club        |1    |5773   |77           |
|2012  |Fodbold Club Midtjylland     |66   |34141  |463          |
|2012  |Reading FC                   |15   |23282  |313          |
|2012  |Clermont Foot 63             |4    |3853   |43           |
|2012  |Spartak Vladikavkaz (-2020)  |16   |14086  |186          |
+------+-----------------------------+-----+-------+-------------+
only showing top 10 rows


In [58]:
club_season_kpis.write.mode("overwrite").parquet(f"{gold_path}/club_season_kpis")

In [59]:
ls /home/gnana/football-lakehouse/gold

club_season_kpis/  fact_player_match/  player_kpis/  player_season_kpis/


In [61]:
#top Scorers
player_kpis.orderBy(col("goals").desc()).limit(10).toPandas()

,player_id,player_name,position,matches_played,total_minutes,goals,assists,yellow_cards,red_cards,goal_involvement,goals_per_90,assists_per_90
0,38253,Robert Lewandowski,Attack,612,51228,507,118,59,1,625,0.891,0.207
1,28003,Lionel Messi,Attack,522,44380,451,224,49,1,675,0.915,0.454
2,8198,Cristiano Ronaldo,Attack,482,41150,432,114,64,4,546,0.945,0.249
3,132098,Harry Kane,Attack,493,39942,346,85,52,0,431,0.780,0.192
4,58864,Pierre-Emerick Aubameyang,Attack,503,38613,294,80,36,1,374,0.685,0.186
5,148455,Mohamed Salah,Attack,526,41706,292,144,20,0,436,0.630,0.311
6,44352,Luis Suárez,Attack,445,35950,289,149,94,0,438,0.724,0.373
7,342229,Kylian Mbappé,Attack,371,28343,273,105,47,1,378,0.867,0.333
8,105521,Ciro Immobile,Attack,501,36595,267,70,80,2,337,0.657,0.172
9,96341,Romelu Lukaku,Attack,540,40816,267,86,47,1,353,0.589,0.190


In [63]:
#minutes sanity
player_kpis.orderBy(col("total_minutes").desc()).limit(10).toPandas()

,player_id,player_name,position,matches_played,total_minutes,goals,assists,yellow_cards,red_cards,goal_involvement,goals_per_90,assists_per_90
0,38253,Robert Lewandowski,Attack,612,51228,507,118,59,1,625,0.891,0.207
1,108390,Thibaut Courtois,Goalkeeper,526,47452,0,1,18,2,1,0.000,0.002
2,59377,David de Gea,Goalkeeper,520,46859,0,3,7,0,3,0.000,0.006
3,121483,Jan Oblak,Goalkeeper,518,46757,0,1,14,1,1,0.000,0.002
4,139208,Virgil van Dijk,Defender,528,46648,50,25,52,4,75,0.096,0.048
5,48015,Lukas Hradecky,Goalkeeper,513,46338,0,0,19,2,0,0.000,0.000
6,59561,Dani Parejo,Midfield,566,46213,79,95,128,2,174,0.154,0.185
7,36139,Dušan Tadić,Midfield,562,45205,185,210,52,0,395,0.368,0.418
8,17259,Manuel Neuer,Goalkeeper,502,45140,0,2,12,1,2,0.000,0.004
9,72476,Bernd Leno,Goalkeeper,493,44552,0,2,21,1,2,0.000,0.004


In [73]:
#goals per 90
player_kpis.orderBy(col("goals_per_90'").desc()).limit(10).toPandas()

,player_id,player_name,position,matches_played,total_minutes,goals,assists,yellow_cards,red_cards,goal_involvement,goals_per_90',assists_per_90'
0,746988,Jesper Uneken,Attack,1,2,1,0,0,0,1,45.000,0.0
1,926945,Shamil Gadzhiev,Midfield,1,2,1,0,0,0,1,45.000,0.0
2,405568,Maurice Malone,Attack,1,3,1,0,0,0,1,30.000,0.0
3,237665,André Ferreira,Attack,1,3,1,0,0,0,1,30.000,0.0
4,697726,Ivan Zazvonkin,Midfield,1,6,1,0,0,0,1,15.000,0.0
5,493674,Jack Lahne,Attack,1,8,1,0,0,0,1,11.250,0.0
6,1045719,Luca Ross,Attack,2,12,1,0,0,0,1,7.500,0.0
7,398728,Kirill Volchkov,Defender,1,13,1,0,1,0,1,6.923,0.0
8,569488,Giannis Sardelis,Midfield,1,13,1,0,0,0,1,6.923,0.0
9,993407,Poyraz Yıldırım,Attack,3,13,1,0,0,0,1,6.923,0.0


In [74]:
#club_season_goals
club_season_kpis.orderBy(col("goals").desc()).limit(10).toPandas()

,season,club_name,goals,minutes,total_players
0,2022,Real Madrid Club de Fútbol,192,76234,1072
1,2021,Liverpool Football Club,189,63806,880
2,2014,Olympiakos Syndesmos Filathlon Peiraios,185,72282,1044
3,2016,Galatasaray Spor Kulübü,178,57949,783
4,2018,Galatasaray Spor Kulübü,176,65509,896
5,2021,Real Madrid Club de Fútbol,170,73717,1006
6,2022,FC Bayern München,168,63468,871
7,2017,Galatasaray Spor Kulübü,164,60930,789
8,2015,Club Atlético de Madrid S.A.D.,162,61965,810
9,2023,Manchester City Football Club,159,65173,917
